In [ ]:


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'animals10:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F59760%2F840806%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240421%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240421T091908Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5aac2f18f05b5d3fbaf53ad9c0a0796297ed12be2bdd8246bb04df0ebac50593812745599e041cb4e21c8a9a4deb064e80dca7b7e0bf5d2bf9b311498307f727835e3c8b4b57bd26527106e467b334befebfb3cb3efd69d41f94e24cb1edb17e5607c117a9e406dd1d76483eb91ad0f8f04195a3a95ede2be105bd6c5bbe469f1caf985ddb685e4ac0f7e363a5d66d647bb5e26c58e3db9d9afa66760566484537fac165de09e91836870bd15f5ccd4776aa665ab0ace4efc3fc4b32e52880fb6e4e9c1ea05b3c48070cf11a0cd87ddb5ec9506f4466837f10bc40243d805890df7a5e5b456508b6fa9070e3d84b8bfa64f7e77b8bd64b35f3b4ce268d95c803,just-one-cat:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1203435%2F2011011%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240421%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240421T091908Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3f3dd5b3bf6f778d3ccce688d2ecfdff8c282105b0f1456859a36f626ce46849cd373428eb95d8ea7c6c44e735ac8c430a5e7c9348000f5340deb4fc4faa1cb06ec846a67e59bcd6fcf50d284312fa754a6d1231ffebb7b63e727d7db942d21b640452e28b3ef0d114d3e906a96499fa2055edf407a9a378f733c57804aadf4cc215a089f8186f65686d011158d159776e696556f9a2e68c7c2103587e7f7531457d246c75ce35f34657bb4a18ea6477b2b1e0e51564d96fa6fb341de36cd893fbb1384a4fef6d9d10d342ce1fb7e3340a0e255965d94bfecbf14fbbffa2c4151f15470815e73074c8103b98d8624aa3c80962343d9a8d85b8afd733251723cf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#!pip install --ignore-installed --upgrade tensorflow==2.4.1 --user

Let's import the necessary libraries for this project.  

In [ ]:
import time
import numpy as np
import os
import path
import pydot
from typing import List, Tuple
from matplotlib.pyplot import imshow
%matplotlib inline
import matplotlib.pyplot as plt
import PIL.Image
import pathlib
import shutil

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model

from tensorflow.python.keras.utils import layer_utils
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG

import scipy.misc

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last') # can be channels_first or channels_last.
K.set_learning_phase(1) # 1 stands for learning phase

In [ ]:
def identity_block(X: tf.Tensor, level: int, block: int, filters: List[int]) -> tf.Tensor:

    # layers will be called conv{level}_iden{block}_{convlayer_number_within_block}'
    conv_name = f'conv{level}_{block}' + '_{layer}_{type}'

    # unpack number of filters to be used for each conv layer
    f1, f2, f3 = filters

    # the shortcut branch of the identity block
    # takes the value of the block input
    X_shortcut = X

    # first convolutional layer (plus batch norm & relu activation, of course)
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1),
               padding='valid', name=conv_name.format(layer=1, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=1, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=1, type='relu'))(X)

    # second convolutional layer
    X = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1),
               padding='same', name=conv_name.format(layer=2, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=2, type='bn'))(X)
    X = Activation('relu')(X)

    # third convolutional layer
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1),
               padding='valid', name=conv_name.format(layer=3, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=3, type='bn'))(X)

    # add shortcut branch to main path
    X = Add()([X, X_shortcut])

    # relu activation at the end of the block
    X = Activation('relu', name=conv_name.format(layer=3, type='relu'))(X)

    return X

In [ ]:
def convolutional_block(X: tf.Tensor, level: int, block: int, filters: List[int], s: Tuple[int,int,int]=(2, 2)) -> tf.Tensor:

    # layers will be called conv{level}_{block}_{convlayer_number_within_block}'
    conv_name = f'conv{level}_{block}' + '_{layer}_{type}'

    # unpack number of filters to be used for each conv layer
    f1, f2, f3 = filters

    # the shortcut branch of the convolutional block
    X_shortcut = X

    # first convolutional layer
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=s, padding='valid',
               name=conv_name.format(layer=1, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=1, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=1, type='relu'))(X)

    # second convolutional layer
    X = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same',
               name=conv_name.format(layer=2, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=2, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=2, type='relu'))(X)

    # third convolutional layer
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name.format(layer=3, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=3, type='bn'))(X)

    # shortcut path
    X_shortcut = Conv2D(filters=f3, kernel_size=(1, 1), strides=s, padding='valid',
                        name=conv_name.format(layer='short', type='conv'),
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=conv_name.format(layer='short', type='bn'))(X_shortcut)

    # add shortcut branch to main path
    X = Add()([X, X_shortcut])

    # nonlinearity
    X = Activation('relu', name=conv_name.format(layer=3, type='relu'))(X)

    return X

In [ ]:
def ResNet50(input_size: Tuple[int,int,int], classes: int) -> Model:

    # tensor placeholder for the model's input
    X_input = Input(input_size)

    ### Level 1 ###

    # padding
    X = ZeroPadding2D((3, 3))(X_input)

    # convolutional layer, followed by batch normalization and relu activation
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               name='conv1_1_1_conv',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='conv1_1_1_nb')(X)
    X = Activation('relu')(X)

    ### Level 2 ###

    # max pooling layer to halve the size coming from the previous layer
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # 1x convolutional block
    X = convolutional_block(X, level=2, block=1, filters=[64, 64, 256], s=(1, 1))

    # 2x identity blocks
    X = identity_block(X, level=2, block=2, filters=[64, 64, 256])
    X = identity_block(X, level=2, block=3, filters=[64, 64, 256])

    ### Level 3 ###

    # 1x convolutional block
    X = convolutional_block(X, level=3, block=1, filters=[128, 128, 512], s=(2, 2))

    # 3x identity blocks
    X = identity_block(X, level=3, block=2, filters=[128, 128, 512])
    X = identity_block(X, level=3, block=3, filters=[128, 128, 512])
    X = identity_block(X, level=3, block=4, filters=[128, 128, 512])

    ### Level 4 ###
    # 1x convolutional block
    X = convolutional_block(X, level=4, block=1, filters=[256, 256, 1024], s=(2, 2))
    # 5x identity blocks
    X = identity_block(X, level=4, block=2, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=3, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=4, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=5, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=6, filters=[256, 256, 1024])

    ### Level 5 ###
    # 1x convolutional block
    X = convolutional_block(X, level=5, block=1, filters=[512, 512, 2048], s=(2, 2))
    # 2x identity blocks
    X = identity_block(X, level=5, block=2, filters=[512, 512, 2048])
    X = identity_block(X, level=5, block=3, filters=[512, 512, 2048])

    # Pooling layers
    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc_' + str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
# set input image parameters
image_size = (64, 64)
channels = 3
num_classes = 10

In [ ]:
model = ResNet50(input_size = (image_size[1], image_size[0], channels), classes = num_classes)

In [ ]:
model.summary()

In [ ]:
# path to desired image set, relative to current working dir
in_folder = os.path.join('..', 'input', 'animals10', 'raw-img')

file_count = []

# get number of images in each folder (images per class)
for fld in os.listdir(in_folder):
    crt = os.path.join(in_folder, fld)

    image_count = len(os.listdir(crt))

    file_count.append(image_count)

    print(f'{crt} contains {image_count} images')

In [ ]:
print(f'Total number of images: {sum(file_count)}')

In [ ]:
os.listdir(os.path.join(in_folder, 'elefante'))[:10]

In [ ]:
data_dir = pathlib.Path(in_folder)

elefante = list(data_dir.glob('elefante/*'))

PIL.Image.open(str(elefante[0]))

In [ ]:
PIL.Image.open(str(elefante[0])).size

In [ ]:
PIL.Image.open(str(elefante[10]))

In [ ]:
PIL.Image.open(str(elefante[10])).size

In [ ]:
# path to output folder, relative to current working dir
out_folder = os.path.join('..', 'output', 'animals10', 'processed')

Helper function for cropping images into a square.

In [ ]:
def square_crop_image(im: PIL.Image) -> PIL.Image:
    width, height = im.size
    new_size = min(width, height)

    # center crop
    left = (width - new_size) / 2
    top = (height - new_size) / 2
    right = (width + new_size) / 2
    bottom = (height + new_size) / 2

    crop_im = im.crop((left, top, right, bottom))
    crop_im = crop_im.convert('RGB')

    return crop_im

In [ ]:
def make_dataset(in_folder, im_per_class):
    # iterate through all folders (there should be one folder per object class)
    for fld in os.listdir(in_folder):
        # create the output folder for processed images for current class
        # delete folder and contents if there is one already
        out = os.path.join(out_folder, fld)
        if os.path.exists(out):
            shutil.rmtree(out)
        os.makedirs(out)

        fld_path = pathlib.Path(os.path.join(in_folder, fld))
        num_images = 0
        for file in list(fld_path.glob('*')):
            # open image, center crop to a square
            # save to the output folder
            with PIL.Image.open(file) as im:
                crop_im = square_crop_image(im)
                crop_im.save(os.path.join(out, str(num_images) + '.jpg'))
                im.close()
            # break when desired number of images
            # has been processed (to keep classes balance)
            num_images = num_images + 1
            if (num_images > im_per_class):
                break

In [ ]:
# get the number of images that will make our classes balanced
im_per_class = min(file_count)

# process input images
make_dataset(in_folder, im_per_class)

In [ ]:
img_height = image_size[1]
img_width = image_size[0]
batch_size = 32

In [ ]:
data_dir = pathlib.Path(out_folder)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset="training",
    label_mode='categorical', # default mode is 'int' label, but we want one-hot encoded labels (e.g. for categorical_crossentropy loss)
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    label_mode='categorical',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

Examine a few processed images before proceesing to the training.

In [ ]:
plt.figure(figsize=(10, 10))

i = 1

for images, labels in train_ds.take(1):
    for (image, label) in zip(images, labels):
        ax = plt.subplot(4, 4, i)
        plt.imshow(image.numpy().astype("uint8"))
        plt.title(class_names[tf.argmax(label, axis=0)])
        plt.axis("off")
        i = i + 1
        if i == 17:
            break
plt.show()

Cropping was not the best solution, but I'll work with that now.


We convert our labels (e.g. right now we have a label of 1 which stands for the class 'cavallo' for example) through one-hot-encoding (our label of 1 from my example thus becomes the array: 0 1 0 0 0 0 0 0 0 0).  


In [ ]:
# use keras functionality for adding a rescaling layer
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

# rescale training and validation sets
norm_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
norm_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Confirm rescaling results look as expected.

In [ ]:
image_batch, labels_batch = next(iter(norm_train_ds))

# get one image
first_image = image_batch[0]

# confirm pixel values are now in the [0,1] range
print(np.min(first_image), np.max(first_image))

### 5.2 Model training

First, choose an optimizer schedule, a loss function and metrics to track during training.

In [ ]:
model.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize
    metrics=['accuracy'] # metrics to monitor
)

Configure the dataset for performance by enabling prefetching. Read more <a href="https://www.tensorflow.org/guide/data_performance">here</a>.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

norm_train_ds = norm_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
norm_val_ds = norm_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Run the following cell to train your model on 2 epochs with a batch size of 32. We also use this to see how long it takes to train for one epoch.

In [ ]:
import time

start = time.time()

model.fit(
    norm_train_ds,
    validation_data=norm_val_ds,
    epochs = 2)

stop = time.time()

print(f'Training took: {(stop-start)/60} minutes')

On my laptop, training for two epochs takes a huge amount of time. Even after half an hour of training, the accuracy on the training set is 24%, which is of course not acceptable in a real world scenario.

On Kaggle, on a CPU, training takes even longer than on my personal laptop. Kaggle CPU is very very slow.

So we'll have to activate Kaggle's GPU and retrain. In the next section we'll see how.

# 6. Training the ResNet model on Kaggle GPU

Configure the model like before.

In [ ]:
model_on_gpu = ResNet50(input_size = (image_size[1], image_size[0], channels), classes = num_classes)
model_on_gpu.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize
    metrics=['accuracy'] # metrics to monitor
)

Make sure we have enabled the GPU.

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

We use a callback to stop the training process when the accuracy is no longer improving.

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", # monitor validation loss (that is, the loss computed for the validation holdout)
        min_delta=1e-2, # "no longer improving" being defined as "an improvement lower than 1e-2"
        patience=10, # "no longer improving" being further defined as "for at least 10 consecutive epochs"
        verbose=1
    )
]

Train the model.

In [ ]:
start = time.time()
with tf.device('/gpu:0'):
    history = model_on_gpu.fit(
        norm_train_ds,
        validation_data=norm_val_ds,
        epochs=40,
        callbacks=callbacks,
    )
stop = time.time()
print(f'Training on GPU took: {(stop-start)/60} minutes')

Let's have a look at how our metrics progressed across epochs for training and validation sets.  
We examine both accuracy and loss to see if we can draw any interesting conclusions out of it.

In [ ]:
print(history.history.keys())

plt.figure(figsize=(12, 10))

# summarize history for accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

# summarize history for loss
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# 7. Making predictions

### 7.1 Let's evaluate our model on the test set.

In [ ]:
#preds = model_on_gpu.evaluate(X_test, Y_test)
preds = model_on_gpu.evaluate(norm_val_ds)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

### 7.2 Let's try a new, unseen image

In [ ]:
import cv2

test_img_path = '../input/just-one-cat/cat_1.jpg'

# read image
img = cv2.imread(test_img_path)

# reorder RGB channels (opencv reads as BGR by default)
RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# resize to the same input image size that we used when defining the model architecture
x = cv2.resize(RGB_img, image_size, interpolation = cv2.INTER_AREA)

print('Input image shape:', x.shape) # sanity check

plt.imshow(RGB_img)

In [ ]:
x = np.expand_dims(x, axis=0) # fake batch size dimension
x = x/255.0 # normalize pixel values to [0,1] interval

preds = model_on_gpu.predict(x) # get model predictions

print("Model's prediction has this format: [p(0), p(1), ... , p(5)] = ")
print(preds)



print("The prediction for this image is: ", class_names[np.argmax(preds)])

In [ ]:
len(preds[0])

for i, p in enumerate(preds[0]):
    print(f'p:{p:.05f}\t{class_names[i]}')

# 8. Overfitting when training our Residual Neural Network

In the previous section, we used Keras Callbacks to stop training at the optimum number of epochs: when the loss for the validation set was no longer improving.  

The stopping happened very early in the training. Why do you think that happened ? Have a look at the images we used from 10 different classes, take a look at our model's structure and try to compute the number of parameters the network has to learn through training. Please leave your comments below regarding what can be the cause of our model not learning anymore after only a a couple of epochs. I believe this discussion is quite beneficial to our learning process.  

Now, what happens if I decide not to stop training at epoch < 10 ? I want to see what I end up with if I keep training for 100 epochs.

In [ ]:
model_on_gpu_40e = ResNet50(input_size = (image_size[1], image_size[0], channels), classes = len(class_names))

model_on_gpu_40e.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize
    metrics=['accuracy'] # metrics to monitor
)

start = time.time()
with tf.device('/gpu:0'):
    history = model_on_gpu_40e.fit(
        norm_train_ds,
        validation_data=norm_val_ds,
        epochs=40,
        #batch_size=64,
    )
stop = time.time()

print(f'Training for 40 epochs on GPU took: {(stop-start)/60} minutes')

In [ ]:
print(history.history.keys())

plt.figure(figsize=(12, 10))

# summarize history for accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

# summarize history for loss
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()